In [243]:
import numpy as np
import scipy as sp

from scipy.spatial import distance_matrix
from scipy.stats import norm

from sklearn.metrics.pairwise import rbf_kernel

In [244]:
def create_sum_matrix(input_list):
    n = len(input_list)
    sum_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            sum_matrix[i, j] = input_list[i] + input_list[j]

    return sum_matrix

In [245]:
input = np.matrix(
[[4.22016824, 9.52059542, 9.06147393, 4.76238876, 8.68718698],
 [3.60131013, 8.88663821, 7.9274555 , 1.77946764, 4.62207242],
 [6.99930763, 6.92595521, 1.5150891 , 2.23769337, 4.32443509],
 [8.59804346, 6.94750078, 9.1130928 , 4.26227738, 7.30300422],
 [3.77543786, 0.43269196, 5.55759746, 2.3849611 , 5.84682024],
 [1.37847937, 6.78821223, 2.67056467, 4.76752098, 3.20457591],
 [2,3,4,1,1],
 [1,3,5,2,1],
 [7,4,9,2,6],
 [4.2, 9.5, 9, 4.7, 8.6]])

In [246]:
k_value = [3, 4]
sig_value = [1, 2]

In [247]:
size = 0
if input.shape[0] > input.shape[1]:
    size = input.shape[0]
else:
    size = input.shape[1]

In [ ]:
def create_sig(input, k_value, sig_value):
    
    distance = sp.spatial.distance_matrix(input, input)
    distance_sorted = distance
    distance_sorted.sort(axis=0)
    distance_sorted = distance_sorted.transpose()
    mean_distance = distance_sorted[:, 1:k_value+1].sum(axis=1) / k_value
    
    sig = create_sum_matrix(mean_distance)
    sig = np.divide(sig, 2)
    sig = np.multiply(sig, sig_value)
    
    return sig

In [249]:
def generate_kernel(input, sig, size):
    kernel = np.zeros((size, size, size))
    kernel = np.asarray(kernel)
    
    for i in range(0, size):
        for j in range(0, size):
            gamma = 1/ (2 * pow(sig[i][j], 2))
            gamma = sig[i][j]
            print(gamma)
            kernel[i][j] = rbf_kernel(np.asarray(input[i]), np.asarray(input[j]), gamma=gamma )
            kernel[i][j] = np.divide(kernel[i][j], ( np.sqrt(2 * np.pi) * sig[i][j]) ) 
    print("--------")
    return kernel

In [250]:
kernel_count = len(k_value) * len(sig_value)

In [ ]:
kernels = np.zeros((kernel_count, size, size))

z = 0
for i in k_value:
    for j in sig_value:
        kernel = generate_kernel(input, create_sig(input, i, j), size)
        kernel = kernel[:, :, 0]
        kernels[z] = kernel
        z = z+1
            

[[3.55254712 4.53842281 5.37642545 4.24385392 4.81673605 4.781378
  4.0804965  4.04491628 4.5073272  3.52903395]
 [4.53842281 5.52429849 6.36230114 5.2297296  5.80261174 5.76725368
  5.06637219 5.03079197 5.49320288 4.51490964]
 [5.37642545 6.36230114 7.20030379 6.06773225 6.64061438 6.60525633
  5.90437483 5.86879461 6.33120553 5.35291229]
 [4.24385392 5.2297296  6.06773225 4.93516072 5.50804285 5.47268479
  4.7718033  4.73622308 5.19863399 4.22034075]
 [4.81673605 5.80261174 6.64061438 5.50804285 6.08092498 6.04556693
  5.34468543 5.30910521 5.77151613 4.79322289]
 [4.781378   5.76725368 6.60525633 5.47268479 6.04556693 6.01020887
  5.30932738 5.27374716 5.73615807 4.75786483]
 [4.0804965  5.06637219 5.90437483 4.7718033  5.34468543 5.30932738
  4.60844588 4.57286566 5.03527658 4.05698334]
 [4.04491628 5.03079197 5.86879461 4.73622308 5.30910521 5.27374716
  4.57286566 4.53728544 4.99969636 4.02140312]
 [4.5073272  5.49320288 6.33120553 5.19863399 5.77151613 5.73615807
  5.03527658 4

In [252]:
weights = np.full((kernel_count), 1/kernel_count)

In [253]:
similarity_matrix = np.zeros((size, size))
for i in range(0, kernel_count):
    similarity_matrix = similarity_matrix + np.multiply(kernels[i], weights[i])